In [ ]:
import numpy as np
## Function for Orifice 

#For forward pass (Things already known prior to calculation is the mdot, M1, P01, T01 i.e upstream parameters)

def main_nozzle_forward(P01, T01, m):
    A_o= (np.pi*25.4**2/4)*10**-6 #Area of orifice
    A_p= (np.pi*50.8**2/4)*10**-6 #Area of pipe

    Cd= 0.5 #Coefficient of discharge, It will be less than 1
    y= 1.4  #gamma value changes with the type of gas and conditions
    #g=9.81 #gravitational constant
    R= 287.058 #for air
    B = A_o/A_p 
    
    P_01=P01
    T_01=T01
    _m=m
    
    P_1=stat_P(P_01) #these equatiions need Mach number
    T_1=stat_T(T_01)
    
    m_c= mass_critical(P_1, T_1)
    
    if _m>m_c:
        print ('Flow rate too high')
        r_f = m_c/m #reduction factor
        return r_f
    elif _m == m_c:
        print ('Flow is choked')
        P_2 = P_1*(2/(y+1))**(y/(y-1))
        P_02 = P_01 + P_2 - P_1 # need to calculate this? or P_02 is P_2 critical itself?
        return (P_02)
    else: #no choking
        A=A_o
        _B=B
        from sympy.solvers import solve
        from sympy import Symbol
        P_2 = Symbol('P_2')
        _P_2 =solve(Cd*(1-(0.333+1.145*(_B**2+0.7*_B**5+ 12*_B**13))*(P_1-P_2)/(y*P_1))*A*(2*rho1*(P_1-P_2))**0.5-_m, P_2)
        #This might give complex results with imaginary part of very small value in -16 orders, find way to eliminate that.
        P_02 = P_01 + P_2 - P_1
        return (P_02) # we can even get M and P2 from here
    
def mass_crtical(P1,T1):
    P_1=P1
    T_1=T1
    y=gamma
    m_c= Cd*_A*P_1*(2/R*T_1)**0.5*(y/(y+1)*(2/(y+1))**(2/(y-1)))*0.5
    return m_c

def stat_P (P0):
    P_0=P0
    P_stat= P0/(1+(y-1)/2*M**2)**(-y/(y-1))
    return P_stat

def stat_T(T0):
    T_0=T0
    T_stat= T_0/((1+(y-1)/2 *M**2)**-1)
    return T_stat

#externally calculated Expansion factor, can be used if required.

def Exp(B, P1, P2):
    _B=B
    P_1=P1
    P_2=P2
    _gamma=gamma
    Y = 1-(0.333+1.145(_B**2+0.7*_B**5+ 12*_B**13))*(P_1-P_2)/(_gamma*P_1)
    return Y

# Externally measured P02

def downstream(m, m_c, P1, P01): #Calculate P02 i.e Stagnation pressure downstream.
    P_1=P1
    P_01=P01
    y=gamma
    
    if m>m_c:
        print ('Flow rate too high')
        r_f = m_c/m #reduction factor
        return r_f
    elif m=m_c:
        print ('Flow is choked')
        P_2 = P_1*(2/(y+1))**(y/(y-1))
        P_02 = P_01 + P_2 - P_1
        return (P_02)
    else:
        A=A_o
        _m=m
        from sympy.solvers import solve
        from sympy import Symbol
        P_2 = Symbol('P_2')
        _P_2 =solve(Cd*(1-(0.333+1.145*(_B**2+0.7*_B**5+ 12*_B**13))*(P_1-P_2)/(gamma*P_1))*A*(2*rho1*(P_1-P_2))**0.5-_m, P_2)
        P_02 = P_01 + P_2 - P_1
        return (P_02)
    
       
    
#For Backward pass ( known parameters are mdot, M2, P02,T02)

#Calculate downstream static pressure (P2)

def main_nozzle_backward(P02, T02, m):
    A_o= (pi*25.4**2/4)*10**-6 #Area of orifice
    A_p= (pi*50.8**2/4)*10**-6 #Area of pipe

    Cd= 0.5 #Coefficient of discharge, It will be less than 1
    y= 1.4  #gamma value changes with the type of gas and conditions
    g=9.81 #gravitational constant
    R= 287.058 #for air
    B = A_o/A_p 
    
    P_02=P02
    T_02=T02
    _m=m
    
    P_2=stat_P(P_02)  #For this mach number downstream should be known prior
    T_2=stat_T(T_02)

    #Calculating critical upstream static pressure P_1c
    P_2c= P_02 # according to wikipedia static pressure downstream is considered critical 
    P_1c = P_2c*(2/(y+1))**(-y/(y-1)) #critical pressure calculated for upstream (but this is consdiered as stagnation in formula)
    #But further equations compare P_1c to static pressures, this is a confusion
    
    #method 1
    _P_1= stag_up(_m, A_p, A_o, P_2, Cd, y, M)
    
    #method2
    #T_0= T_02
    # _P_1= P1backward(_m, A_p, A_o, T_0, Cd, y ) # why this has 1 term less? Because the mach number is already included in T_0 calculation done prior
    
    if (_P_1<= P_1c):  #Unchoked condition
        P_1=_P_1
        P_01 = P_02 + P_1 - P_2
    else:  #choked condition
        P_1 = P_1c # or should it be directly the stagnation pressure?
        P_01 = P_1
    return P_01
    
    
def stag_up(m, A_1, A, P_2, Cd, y, M):
    #Unchocked upstream static pressure direct method
    _m=m
    rho1=_m/(A_1*M*c)
    _B=A/A_1
    P_1 = Symbol('P_1')
    _P_1 =solve(Cd*(1-(0.333+1.145*(_B**2+0.7*_B**5+ 12*_B**13))*(P_1-P_2)/(y*P_1))*A*(2*rho1*(P_1-P_2))**0.5-_m, P_1)
    return _P_1
    
# Calculation of unchoked upstream static pressure in terms of rho1

def P1backward (m, A_1, A, T_0, Cd, y): #Stagnation temperature T_0, upstream or downstream? My view: Upstream, since rho is for upstream
    
    G = (y-1)/(2*y)*(m/A_1)**2  # A_1 is area of upstream section
    D = m**2/(A_1**2*y*R*T_0) # T_0 is stagnation pressure at inlet
    C_A= Cd*A # where A is effective area of throttling device
    rho1 = Symbol('rho1')
    _rho1 =solve((m/(C_A*(2*(rho1**2*R*T_0-G-rho1*P_2))**0.5))-(1-D/y*(1-P_2/(rho1*R*T_0-G/rho1))), rho1)

   #Now will use rho1 value to calculate P1unchoked
    _P_1 = rho1*R*T_0-(y-1)/(2*y)*(m**2/A_1)*1/rho1
    return _P_1
    












7


In [88]:
import numpy as np
import cmath as cm

from sympy.solvers import solve
from sympy import Symbol
P_2 = Symbol('P_2')
Cd=0.8
_B=0.5
rho1= 40 #approx for air at room temperature and pressure
gamma= 1.4 #Value changes with the type of gas and conditions
g=9.81 #gravitational constant
R= 287.058
P_1= 100*10**5
A = (np.pi*25.4**2/4)*10**-6
m = 0.03
y=gamma



_P_2 =solve(Cd*(1-(0.333+1.145*(_B**2+0.7*_B**5+ 12*_B**13))*(P_1-P_2)/(gamma*P_1))*A*(2*rho1*(P_1-P_2))**0.5-m, P_2)

_P_2_1= _P_2
print (_P_2_1)
_P_2_1real= _P_2_1 
print (_P_2_1real)


[-115493.587427599 - 0.e-18*I, 99993.0206193672 + 0.e-19*I]
[-115493.587427599 - 0.e-18*I, 99993.0206193672 + 0.e-19*I]


In [132]:
import numpy as np
from sympy.solvers import solve
from sympy import Symbol
import cmath as cm

A_o= (np.pi*25.4**2/4)*10**-6 #Area of orifice
A_p= (np.pi*50.8**2/4)*10**-6 #Area of pipe

Cd= 0.8 #Coefficient of discharge, It will be less than 1
y= 1.4  #gamma value changes with the type of gas and conditions
#g=9.81 #gravitational constant
R= 287.058 #for air
B = np.sqrt(A_o/A_p) 
#M=0.8
m=0.3

def stat_P (P0):
    P_0=P0
    P_stat= P0/(1+(y-1)/2*M**2)**(-y/(y-1))
    return P_stat

def stat_T(T0):
    T_0=T0
    T_stat= T_0/((1+(y-1)/2 *M**2)**-1)
    return T_stat

def stag_up(m, A_1, A, P_2, Cd, y, M):
    #Unchocked upstream static pressure direct method
    _m=m
    rho1=_m/(A_1*M*c)
    _B=A/A_1
    P_1 = Symbol('P_1')
    _P_1 =solve(Cd*(1-(0.333+1.145*(_B**2+0.7*_B**5+ 12*_B**13))*(P_1-P_2)/(y*P_1))*A*(2*g*rho1*(P_1-P_2))**0.5-_m, P_1)
    return _P_1
    
# Calculation of unchoked upstream static pressure in terms of rho1

def P1backward (m, A_1, A, T_0, Cd, y): #Stagnation temperature T_0, upstream or downstream? My view: Upstream, since rho is for upstream
    
    G = (y-1)/(2*y*g)*(m/A_1)**2  # A_1 is area of upstream section
    D = m**2/(A_1**2*g*y*R*T_0) # T_0 is stagnation pressure at inlet
    C_A= Cd*A # where A is effective area of throttling device
    rho1 = Symbol('rho1')
    _rho=solve((m/(C_A*(2*(rho1**2*R*T_0-G-rho1*P_2))**0.5))-(1-D/y*(1-P_2/(rho1*R*T_0-G/rho1))), rho1)

    _rho1 = _rho[2]  #Need to edit this as it is taking only one input value from the list
    print (_rho)
   #Now will use rho1 value to calculate P1unchoked
    _P_1 = _rho1*R*T_0-(y-1)/(2*y)*(m**2/A_1)*1/_rho1
    return _P_1

P_2 = 100000
Tstat= 298
c= 340
#T_0 = Tstat/(1+(y-1)/2 *M**2)
T_0=250

_P_1_1= stag_up(m,A_p,A_o,P_2,Cd, y,M)
_P_1_2= P1backward(m, A_p, A_o, T_0, Cd, y)

print (_P_1_1)
print (_P_1_2)

#Find a way that it only takes in positive solution


[-0.286821959868816, -0.0582216000336342, 1.63516587930384]
[165425.334989757, -22649.6466476831 - 22156.820340231*I, -22649.6466476831 + 22156.820340231*I]
117346.466290966


In [90]:
import numpy as np
from scipy.optimize import fsolve

_B=0.5
Cd=0.8
P_1=10000000
A=(np.pi*25.4**2/4)*10**-6
rho1=40
y=1.4
_m=0.0003

def equation(P_2):
    return ((Cd*(1-(0.333+1.145*(_B**2+0.7*_B**5+ 12*_B**13))*(P_1-P_2)/(y*P_1))*A*(2*rho1*(P_1-P_2))**0.5)-_m)

P_2 = fsolve (equation, 1000000) #xtol=1)

print (P_2, equation(P_2))


#Solution = sp.fsolve(Cd*(1-(0.333+1.145*(_B**2+0.7*_B**5+ 12*_B**13))*(P_1-P_2)/(y*P_1))*A*(2*g*rho1*(P_1-P_2))**0.5, 10000000,xtol=1,)

[-11672571.85344858] [-5.89410248e-15]


In [60]:
import numpy as np
from scipy.optimize import fsolve
def f(x):
    return (x**2+4*x+4)

x= fsolve(f, 0, xtol= 0.1)
x,f(x)
    

(array([-1.99999999]), array([0.]))

In [131]:
import numpy as np
from scipy.optimize import fsolve

A_o= (np.pi*25.4**2/4)*10**-6 #Area of orifice
A_p= (np.pi*50.8**2/4)*10**-6 #Area of pipe

Cd= 0.5 #Coefficient of discharge, It will be less than 1
y= 1.4  #gamma value changes with the type of gas and conditions
g=9.81 #gravitational constant
R= 287.058 #for air
B = np.sqrt(A_o/A_p) 
P_2= 100000
A=(np.pi*25.4**2/4)*10**-6
#rho1=40
y=1.4
_m=0.03 
A_1=A_p
T_0= 250

G = (y-1)/(2*y*g)*(_m/A_1)**2  # A_1 is area of upstream section
D = _m**2/(A_1**2*g*y*R*T_0) # T_0 is stagnation pressure at inlet
C_A= Cd*A

def equation(rho1):
    #print ((2*g*(rho1**2*R*T_0-G-rho1*P_2))**0.5)
    return (-(_m/(C_A*(2*g*(rho1**2*R*T_0-G-rho1*P_2))**0.5))+(1-D/y*(1-P_2/(rho1*R*T_0-G/rho1))))

rho1= fsolve (equation, 20, xtol=0.000001)
print ('rho1=',rho1,'error=', equation(rho1))
#print (P_2, equation(P_2))

_P_1 = rho1*R*T_0-(y-1)/(2*g*y)*(_m**2/A_1)*1/rho1
print ('P1 upstream =',_P_1)

#Why only for _m=0.03, and initial guess of rho1 = 20 we get good results?



[22890.34148729]
[22890.34148729]
[22890.34148729]
[22890.34184116]
[2350294.52803971]
[1209504.95548113]
[22890.34184116]
[2350294.52803971]
[1209504.95548113]
[570394.11851228]
[319554.19064441]
[125417.00369031]
[97064.24465898]
[14151.85161412]
[88334.61068422]
[22890.34148729]
[14151.85141486]
[7080.71892371]
[6916.25087921]
[13988.23775903]
[6916.25099615]
[6068.83191812]
[4366.09484997]
[696.85548024]
[5582.65531625]
[4366.09484997]
[696.85552435]
[679.56652399]
[644.16681887]
[569.35706634]
[392.86778916]
[nan]
[nan]
[392.86784964]
[370.52794791]
[322.03837927]
[196.5919112]
[nan]
[nan]
[196.59201919]
[185.36218313]
[160.68286571]
[94.29726732]
[131.18728567]
[122.1109396]
[117.78873932]
[118.44084231]
[118.41183866]
[118.41160984]
[118.41160984]
rho1= [1.40058836] error= [-7.10582371e-10]
P1 upstream = [100512.51861085]


C:\Users\jyogiy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in sqrt
C:\Users\jyogiy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in sqrt


In [150]:
import numpy as np

def ratio(P_01, P_02, m): # P_01 stagnation pressure from calculations through pipe 1
    m_old=m
    del_P= P_02-P_01
    c= 1e-5 #Value to be decided on how fast convergence is needed
    if del_P < 0.01: #to be decided as per requirement
        print("Values calculated are correct for this time step")
        m_new = m_old
        P_avg= (P_02+P_01)/2
        return (m_new, P_avg)
    else:
        m_new = m_old*np.exp(c*del_P)
        return (m_new)

ratio (100000, 11000, 0.002)

        
        

Values calculated are correct for this time step


(0.002, 55500.0)

In [136]:
#For backward pass dividing flows at junction

import numpy as np

#Assume 50% division at every node in the beginning
def divide(ratio2, P_01, P_02):
    del_P = P_02-P_01
    rat_old= ratio2
    c1=0.00005
    c2=0.00005
    
    if del_P > 0.001:
        rat_new = rat_old* np.exp(-c1*del_P)
    elif del_P < -0.001:
        rat_new = 1 - np.exp(c2*del_P) + rat_old*np.exp(c2*del_P)
    else:
        print ('The ratio is correct')
        rat_new = rat_old
    
    return rat_new

ratio1 = 1 - divide(0.4,200000,210000)
# calculate new stagnation pressure and the process continues till we get delta P very small
#ratio2 = 1 - divide(ratio1,)


        
        
    
    
    

0.2426122638850534

In [149]:
#Addition of masses at node for total mass afterwards

def add_m(m1,m2):
    m_t=m1+m2
    return m_t

#Division of masses during bacward pass calculation
def m_next(m_t,n): #where n is the number of divsion of input pipeline
    m_divide = m_t/n
    return m_divide

print (add_m (2,5))
m_next (10,2)

7


5.0